In [1]:
import pandas as pd

import json, csv
from csv import writer, reader
import os

def save2json(filename, dump):
    out_file = open(filename, "w")
    json.dump(dump, out_file, indent=6)
    out_file.close()


def save2csv(filename, header, data):
    with open(filename, "w", newline="") as csvfile:
        csvw = csv.writer(
            csvfile, delimiter=",", quotechar="|", quoting=csv.QUOTE_MINIMAL
        )
        csvw.writerow(header)
        for d in data:
            csvw.writerow(d)

In [2]:
def get_textual_entities(infile, entity_file, outfile):
    memes_entities = []
    cache = []
    errors = []
    with open(infile) as csvfile:
        urls = reader(csvfile)
        # Loading file with dbpedia entities
        with open(entity_file) as efile:
            data = json.load(efile)
            # Mapping DBPedia entities to Wikidata
            with open("/opt/airflow/notebooks/data/dbpedia-wikidata.json") as jsonfile:
                db2wdmapping = json.load(jsonfile)
                for row in urls:
                    if row[1] in data.keys() and not row[1] in cache:
                        cache.append([row[1]])
                        newdata = data[row[1]]
                        newdata["url"] = row[1]
                        if "Resources" in newdata.keys():
                            for r in newdata["Resources"]:
                                res = r["@URI"].replace(
                                    "http://dbpedia.org/resource/", ""
                                )
                                if res in db2wdmapping.keys():
                                    r["QID"] = (
                                        "https://www.wikidata.org/wiki/"
                                        + db2wdmapping[res]
                                    )
                                if type(r["@types"]) == str:
                                    ls = (
                                        r["@types"]
                                        .replace(
                                            "DUL:",
                                            "http://www.loa-cnr.it/ontologies/DOLCE-Lite#",
                                        )
                                        .replace(
                                            "Wikidata:",
                                            "https://www.wikidata.org/wiki/",
                                        )
                                        .replace("Schema:", "https://schema.org/")
                                        .split(",")
                                    )
                                    ls = list(filter(None, ls))
                                    ls = [l for l in ls if "DBpedia" not in l]
                                # elif(type(r["@types"])==list):
                                # ls = list(map(lambda s: s.replace("DUL:", "http://www.loa-cnr.it/ontologies/DOLCE-Lite#").replace("Wikidata:", "https://www.wikidata.org/wiki/").replace("Schema:", "https://schema.org/").replace("DBpedia:", ""),r["@types"]))
                                # errors.append([row[1],res,r["@types"]])
                                # Removing dbpedia types that do not have a correspondence in wikidata
                                for l in ls:
                                    if l in db2wdmapping.keys():
                                        l = (
                                            "https://www.wikidata.org/wiki/"
                                            + db2wdmapping[l]
                                        )
                                r["@typeList"] = ls
                        memes_entities.append(data[row[1]])
    save2json(outfile, memes_entities)
    save2csv("errors.csv", ["kym", "entity", "@types"], errors)

In [3]:
spotlight_enrichment_raw = "/opt/airflow/notebooks/baseData/kym.spotlight.raw.json"

get_textual_entities(
    "/opt/airflow/notebooks/data/kym.media.frames.csv",
    spotlight_enrichment_raw,
    "/opt/airflow/notebooks/data/kym.media.frames.textual.enrichment.json",
)